# Building Complex Models using function API

The function api allows one to build neural networks with more complex topologies with multipla inputs or outputs

***Nonsequential neural networks [wide & deep neural network]**

this architecturw makes it possible for the neural network to learn both deep patterns using the deep path and simple rules through the short path. 

context:
A regular MLP forces all the data to flow through the fullstack of layers, thus simple patterns in the data may end up being distorted by this sequence of transformations.let's test it out

In [3]:
from sklearn.datasets import fetch_california_housing
import tensorflow as tf
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [4]:
# fetch the data
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import tensorflow as tf

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [5]:
normalization_layer = tf.keras.layers.Normalization()

hidden_layer1 = tf.keras.layers.Dense(30, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1)

input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
normalized = normalization_layer(input_)

hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)

model = tf.keras.Model(inputs=[input_], outputs=[output])


In the above code:

- step 1:- created 5 layers with the `Normalization layer` to standardize the inputs. {incase you are wondering, relu is an activation function}
- step 2: created an `input object` i.e `_input`:- this is a specification of the kind of input the model will get includin it's `shape` and optionally its `dtype`. 
- step 3:- we call the `normalization layer` as a function and passing it the input object. it tells keras how yo connect the layers.
- step 4: In the same way, we then pass `normalized` to `hidden_layer1`, which outputs `hidden1`, and we pass `hidden1` to `hidden_layer2`, which outputs `hidden2`.
- step 5:- we use the `concat_layer` to concatenate the input and the second hidden layer’s output
- step 6: pass  `concat` to the` output_layer`, which gives us the final output.
- step 7:- create a Keras `Model`, specifying which inputs and outputs to use.


In an instance where we would want to send asubset of the features through the wide path, and a different subset through the deep path., we can use **multiple inputs**

for example, suppose we want to send 5 features through the wide path, (feature 0 to 4) and 6 features throuh the deep path(feature 2 to 7), we can achieve this as follows;

In [8]:
input_wide = tf.keras.layers.Input(shape=[5])
input_deep = tf.keras.layers.Input(shape=[6])

norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()

norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)

concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)

model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])


There are a few things to notice in the above code, compared to the previous one:

- Each `dense` layer is created and called on the same line. strategy to make the code more concise
- `tf.keras.layers.concatenate()` is used to create a `concatenate` layer and calls it with the given inputs.
- `inputs=[input_wide, input_deep]` was specified when creating the model sice there were tow inputs.


the model can be compiled as usual but when calling the `fit()` methof, instead of passing a single input matrix `X_train`, pair matrices must be passed `(X_train_wide, X_train_deep)` one per input.
 the same is true for `X_valid` and also for `X_test` and `X_new` when you call `evaluate()` or `predict()`

In [13]:
# initialize the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

# training the model
history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20,
                    validation_data=((X_valid_wide, X_valid_deep), y_valid))
mse_test = model.evaluate((X_test_wide, X_test_deep), y_test)
y_pred = model.predict((X_new_wide, X_new_deep))
        

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 442us/step - RootMeanSquaredError: 1.5601 - loss: 2.5308 - val_RootMeanSquaredError: 0.8676 - val_loss: 0.7527
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 297us/step - RootMeanSquaredError: 0.7602 - loss: 0.5793 - val_RootMeanSquaredError: 0.9316 - val_loss: 0.8679
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 289us/step - RootMeanSquaredError: 0.6746 - loss: 0.4556 - val_RootMeanSquaredError: 0.6486 - val_loss: 0.4207
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 292us/step - RootMeanSquaredError: 0.6467 - loss: 0.4185 - val_RootMeanSquaredError: 0.6886 - val_loss: 0.4742
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 289us/step - RootMeanSquaredError: 0.6316 - loss: 0.3992 - val_RootMeanSquaredError: 0.5901 - val_loss: 0.3483
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 301us/step - RootMeanSquaredError: 0.6210 - loss: 0.3859 - val_RootMeanSquaredError: 0.8969 - val_loss: 0.8044
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 296us/step - RootMeanSquaredError

In [ ]:
# E
    